## Demo for Pytorch implementation of baseline

In [1]:
#Download

from google.colab import auth
auth.authenticate_user()

!gsutil cp  gs://homework2data/mask.zip  .
!unzip -q mask.zip
!rm mask.zip

!gsutil cp  gs://homework2data/text_objseg_pretrained_torch_converted_with_lstm  .


KeyboardInterrupt: ignored

In [0]:
!gsutil cp  gs://homework2data/image.zip  .
!unzip -q image.zip
!rm image.zip


In [15]:
import sys
import os
print (os.getcwd())
from model import ImgSegRefExpModel
import torch
import sys
import skimage.io
import numpy as np
import tensorflow as tf
import json
import timeit
#import data
from data import ImageSegmentationDataset
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision


/content


In [0]:
#Utils

def load_vocab_dict_from_file(dict_file):
    with open(dict_file) as f:
        words = [w.strip() for w in f.readlines()]
    vocab_dict = {words[n]:n for n in range(len(words))}
    return vocab_dict

#Util functions

# # all boxes are [num, height, width] binary array
def compute_mask_IU(masks, target):
    #print (np.sum(np.logical_and(masks, target)))
    assert(target.shape[-2:] == masks.shape[-2:])
    I = torch.sum(np.logical_and(masks, target))
    U = torch.sum(np.logical_or(masks, target))
    return I, U

def resize_and_pad(im, input_h, input_w):
    # Resize and pad im to input_h x input_w size
    im_h, im_w = im.shape[:2]
    scale = min(input_h / im_h, input_w / im_w)
    resized_h = int(np.round(im_h * scale))
    resized_w = int(np.round(im_w * scale))
    pad_h = int(np.floor(input_h - resized_h) / 2)
    pad_w = int(np.floor(input_w - resized_w) / 2)

    resized_im = skimage.transform.resize(im, [resized_h, resized_w])
    if im.ndim > 2:
        new_im = np.zeros((input_h, input_w, im.shape[2]), dtype=resized_im.dtype)
    else:
        new_im = np.zeros((input_h, input_w), dtype=resized_im.dtype)
    new_im[pad_h:pad_h+resized_h, pad_w:pad_w+resized_w, ...] = resized_im

    return new_im

def resize_and_crop(im, input_h, input_w):
    # Resize and crop im to input_h x input_w size
    im_h, im_w = im.shape[:2]
    scale = max(input_h / im_h, input_w / im_w)
    resized_h = int(np.round(im_h * scale))
    resized_w = int(np.round(im_w * scale))
    crop_h = int(np.floor(resized_h - input_h) / 2)
    crop_w = int(np.floor(resized_w - input_w) / 2)

    resized_im = skimage.transform.resize(im, [resized_h, resized_w])
    if im.ndim > 2:
        new_im = np.zeros((input_h, input_w, im.shape[2]), dtype=resized_im.dtype)
    else:
        new_im = np.zeros((input_h, input_w), dtype=resized_im.dtype)
    new_im[...] = resized_im[crop_h:crop_h+input_h, crop_w:crop_w+input_w, ...]

    return new_im


In [0]:
# trained model

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

root = '/content/'

pretrained_model_file = root + 'text_objseg_pretrained_torch_converted_with_lstm'

image_dir = root + 'images/'
mask_dir = root + 'mask/'
query_file = root + 'referit_query_test.json'
bbox_file = root + 'referit_bbox.json'
imcrop_file = root + 'referit_imcrop.json'
imsize_file = root + 'referit_imsize.json'
vocab_file = root + 'vocabulary_referit.txt'

In [4]:
# Load vocabulary
vocab_dict = load_vocab_dict_from_file(vocab_file)

# Load model and weights
model = ImgSegRefExpModel(mlp_hidden=500, vocab_size=8803, emb_size=1000, lstm_hidden_size=1000).to(device)
pre_trained = torch.load(pretrained_model_file)
model.load_state_dict(pre_trained)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:
set(model.state_dict().keys()).difference(set(pre_trained.keys()))

set()

In [6]:
#saving image sizes

query_dict = json.load(open(query_file))

image_names = set()
for key, value in query_dict.items():
    image_names.add(key.split('_')[0])

print (len(image_names))
import skimage.io

image_sizes = {}
for name in image_names:
    im = skimage.io.imread(image_dir + name + '.jpg')
    image_sizes[name] = im.shape

print (image_sizes)

9994
{'17147': (360, 480, 3), '31502': (320, 480, 3), '9232': (480, 360, 3), '15752': (360, 480, 3), '14552': (480, 360, 3), '39287': (480, 360, 3), '37582': (320, 480, 3), '4302': (360, 480, 3), '1417': (360, 480, 3), '32338': (360, 480, 3), '4502': (360, 480, 3), '16146': (360, 480, 3), '39218': (360, 480, 3), '11057': (480, 360, 3), '31945': (360, 480, 3), '2313': (360, 480, 3), '21891': (360, 480, 3), '1801': (360, 480, 3), '22241': (480, 360, 3), '3090': (360, 480, 3), '38774': (360, 480, 3), '8090': (360, 480, 3), '3353': (360, 480, 3), '8168': (480, 360, 3), '26133': (480, 360, 3), '13454': (360, 480, 3), '14879': (360, 480, 3), '35877': (360, 480, 3), '37353': (320, 480, 3), '10378': (360, 480, 3), '40194': (480, 360, 3), '3381': (360, 480, 3), '39168': (360, 480, 3), '30520': (320, 480, 3), '37180': (480, 320, 3), '12650': (480, 360, 3), '1226': (360, 480, 3), '1762': (360, 480, 3), '21528': (360, 480, 3), '37594': (320, 480, 3), '3174': (360, 480, 3), '40429': (360, 480, 3), 

In [7]:
for key, value in image_sizes.items():
  if (len(value) != 3):
    print (key, len(value), value)

8103 2 (480, 360)


In [0]:
batch_size = 3
test_dataset = ImageSegmentationDataset(query_file, image_dir, mask_dir)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [21]:
log_file = open('log.txt', 'a+')
sum_I = 0
sum_U = 0
model.eval()
import time
import torchvision.transforms as transforms



total_images = len(test_loader)

print ('Total batches', total_images)
for batch_idx, (image, text, mask, original_image) in enumerate(test_loader):
    batch_time = time.time()
    with torch.no_grad():
      text = text.long()
      model_in = time.time()
      output_mask = model((image.to(device), text.to(device)))
      print ('Model time', time.time() - model_in)
      output_mask = output_mask.squeeze(1)
      output_mask = output_mask.squeeze(1)
      #prediction = output_mask.cpu().detach().numpy() > 0
      #print ('o shape', output_mask.shape)
      looping_time_start = time.time()
      for i in range(batch_size):
        image_name = ""
        for digit in original_image[i]:
            if (digit == -1):
              break
            image_name += str(digit.item())
        if (len(image_sizes[image_name]) == 3):
          h, w, _ = image_sizes[image_name]
        else:
          h, w = image_sizes[image_name]
        
        #labels = transforms.CenterCrop(mask[i] > 0, h, w).astype(torch.bool)
        labels = resize_and_crop(mask[i].cpu().detach().numpy() > 0, h, w).astype(np.bool)
        labels = torch.from_numpy(labels > 0)
        #labels = labels > 0
        #print ('l and P', labels.shape)
        prediction = torch.from_numpy(resize_and_crop(output_mask[i].cpu().detach().numpy() > 0, h, w).astype(np.bool))
        #print ('l and P', labels.shape, prediction.shape)
        I, U = compute_mask_IU(prediction, labels)
        I = float(I)
        U = float(U)
        sum_I += I
        sum_U += U
        IoU = I / U
      
      looping_time_end = (time.time())
      
      print ('Loop Time', looping_time_end - looping_time_start )
        #print (IoU
  #     log_file.write(image_name + ',' + str(IoU) + '\n')
  #     plt.figure(figsize=(12, 6))
  #     plt.subplot(1, 2, 1)
  #     plt.imshow(np.array(im))
  #     plt.subplot(1, 2, 2)
  #     plt.imshow(prediction)    
      
      print ('Batch ID', batch_idx, 'IOU so far', float(sum_I)/sum_U, 'Time', time.time() - batch_time)
  



#print ('Final IOU', float(sum_I)/sum_U)


Total batches 20035
Get item time 0.05962562561035156
Get item time 0.058298349380493164
Get item time 0.0601503849029541


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Model time 0.2214968204498291
Loop Time 0.22886133193969727
Batch ID 0 IOU so far 0.278464385699386 Time 0.4515361785888672
Get item time 0.054367780685424805
Get item time 0.05347728729248047
Get item time 0.06422686576843262
Model time 0.15044879913330078
Loop Time 0.2117924690246582
Batch ID 1 IOU so far 0.25753174561510955 Time 0.36318397521972656
Get item time 0.05771684646606445
Get item time 0.05472612380981445
Get item time 0.05943751335144043
Model time 0.15055537223815918


KeyboardInterrupt: ignored

In [0]:
 for batch_idx, (image, text, mask, original_image) in enumerate(test_loader):
    print ('Image Number:', batch_idx)
    text = text.long()
    output_mask = model((image, text))
    #output_mask = model((image, text))
    
    print ('M', mask.shape)
    mask = mask.squeeze(0)
    labels = mask > 0
    #labels = mask.cpu().detach().numpy() > 0
    print("Mask shape", output_mask.shape)
    output_mask = output_mask.squeeze(0)
    output_mask = output_mask.squeeze(0)
    print("Mask shape squeezed", output_mask.shape)
    
    image_name = ""
    for digit in original_image[0]:
        #print (digit.item())
        image_name += str(digit.item())

    #print (int (q))
    
    im = skimage.io.imread('/Users/shubhammehrotra/text_objseg/exp-referit/referit-dataset/images/' + image_name + '.jpg')
    
    prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * (512,512)).astype((np.float32))
    # Final prediction
    
    #prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)
    #print (np.sum(np.logical_and(output_mask.cpu().detach().numpy(), labels)))
    print ('prediction', prediction.shape)
    print ('Labels', labels)
    
    #prediction = im_processing.resize_and_crop(prediction, im.shape[0], im.shape[1])
    print (type(prediction))
    I, U = eval_tools.compute_mask_IU(prediction, labels)
    I = float(I)
    U = float(U)
    sum_I += I
    sum_U += U
    print ('Image IOU', I / U)
    print ('Updated IOU', sum_I / sum_U)
    print ('\n')
    break

print ('Final IOU', float(sum_I)/sum_U)



#     print('testing image %d / %d' % (n_im, num_im))
#     imname = imlist[n_im]

#     # Extract visual features from all proposals
#     im = skimage.io.imread(image_dir + imname)
#     processed_im = skimage.img_as_ubyte(im_processing.resize_and_pad(im, input_H, input_W))
#     if processed_im.ndim == 2:
#         processed_im = np.tile(processed_im[:, :, np.newaxis], (1, 1, 3))

#     imcrop_val[...] = processed_im.astype(np.float32) - segmodel.vgg_net.channel_mean
#     for imcrop_name, _, description in flat_query_dict[imname]:
#         mask = load_gt_mask(mask_dir + imcrop_name + '.mat').astype(np.float32)
#         labels = (mask > 0)
#         processed_labels = im_processing.resize_and_pad(mask, input_H, input_W) > 0

#         text_seq_val[:, 0] = text_processing.preprocess_sentence(description, vocab_dict, T)
#         scores_val = sess.run(scores, feed_dict={
#                 text_seq_batch  : text_seq_val,
#                 imcrop_batch    : imcrop_val
#             })
#         scores_val = np.squeeze(scores_val)

#         # Evaluate the segmentation performance of using bounding box segmentation
#         pred_raw = (scores_val >= score_thresh).astype(np.float32)
#         predicts = im_processing.resize_and_crop(pred_raw, im.shape[0], im.shape[1])
#         I, U = eval_tools.compute_mask_IU(predicts, labels)
#         cum_I += I
#         cum_U += U
#         this_IoU = I/U
#         for n_eval_iou in range(len(eval_seg_iou_list)):
#             eval_seg_iou = eval_seg_iou_list[n_eval_iou]
#             seg_correct[n_eval_iou] += (I/U >= eval_seg_iou)
#         seg_total += 1



In [0]:
#Displaying output
   
im = skimage.io.imread('/Users/shubhammehrotra/text_objseg/exp-referit/referit-dataset/images/' + '35728' + '.jpg')

    # Final prediction
prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)
#print (prediction)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(prediction)
#print (labels.shape, prediction.shape)
#print ()
prediction = im_processing.resize_and_crop(prediction, im.shape[0], im.shape[1])
I, U = eval_tools.compute_mask_IU(prediction, labels)

print ('IOU',  I/U)


#         # Evaluate the segmentation performance of using bounding box segmentation
#         pred_raw = (scores_val >= score_thresh).astype(np.float32)
#         predicts = im_processing.resize_and_crop(pred_raw, im.shape[0], im.shape[1])
#         I, U = eval_tools.compute_mask_IU(predicts, labels)
#         cum_I += I
#         cum_U += U
#         this_IoU = I/U

#print("query text = '%s'" % query)



In [0]:
original_image_name = "12346"
char = []
    
char = [int(i) for i in original_image_name]

char = torch.from_numpy(np.array(char))

q = ""
for digit in s:
    print (digit)
    q += str(digit)

print (int (q))



### Run on sample image

In [0]:
# input image and query text
im_file = './38100.jpg'
query = 'sky above the bridge'

In [0]:
# Load the model
# Model Param
T = 20
N = 1
input_H = 512; featmap_H = (input_H // 32)
input_W = 512; featmap_W = (input_W // 32)
num_vocab = 8803
embed_dim = 1000
lstm_dim = 1000
mlp_hidden_dims = 500

channel_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)

# Run on the input image and query text
text_seq_val = np.zeros((T, N), dtype=np.float32)
imcrop_val = np.zeros((N, input_H, input_W, 3), dtype=np.float32)

# Preprocess image and text
im = skimage.io.imread(im_file)

# Makes it uint8
processed_im = skimage.img_as_ubyte(resize_and_pad(im, input_H, input_W))
print(processed_im.dtype)

imcrop_val[0, :] = processed_im.astype(np.float32) - channel_mean
print(imcrop_val.dtype)

# preprocess sentence pads 0's before the phrase. flipping it ruins the output
text_seq_val[:, 0] = preprocess_sentence(query, vocab_dict, T)
print(text_seq_val.dtype)
print (text_seq_val)

imcrop_val = torch.Tensor(imcrop_val).permute(0, 3, 1, 2) 
text_seq_val = torch.LongTensor(text_seq_val).t() 

In [0]:
print(imcrop_val.shape)
print(text_seq_val.shape)

In [0]:
# Forward pass to get response map
output_mask = model((imcrop_val, text_seq_val))

print("Mask shape", output_mask.shape)
output_mask = output_mask.squeeze(0)
output_mask = output_mask.squeeze(0)
print("Mask shape squeezed", output_mask.shape)

# Final prediction
prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)

In [0]:
prediction

In [0]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(prediction)

print("query text = '%s'" % query)